## Azure의 Computer Vision 활용

In [1]:
import gradio as gr
import requests
from PIL import Image, ImageDraw, ImageFont
import random
import platform
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
ENDPOINT = os.getenv("ENDPOINT")
API_KEY = os.getenv("API_KEY")

#### Caption 연동 테스트

In [2]:
FEATURES = "caption"

def request_vision(features, image_path):
    endpoint = f"{ENDPOINT}computervision/imageanalysis:analyze?api-version=2024-02-01&features={features}"

    headers = {
        "Ocp-Apim-Subscription-Key":API_KEY,
        "Content-Type":"application/octet-stream"
    }

    with open(image_path, "rb") as image_file:
        image_data = image_file.read()

    response = requests.post(endpoint, headers=headers, data=image_data)
    print(response.text)
    if response.status_code == 200:
        return response.json()
    else:
        return {}

request_vision(FEATURES, "C:/Users/jooeu/Desktop/git/ms-ai-school/250324_AIVision/test2.jpg")

{"modelVersion":"2023-10-01","captionResult":{"text":"a woman in a white sweater and hat","confidence":0.7546942830085754},"metadata":{"width":542,"height":512}}


{'modelVersion': '2023-10-01',
 'captionResult': {'text': 'a woman in a white sweater and hat',
  'confidence': 0.7546942830085754},
 'metadata': {'width': 542, 'height': 512}}

#### Caption을 활용해서 이미지 크롭하기
computer vision의 caption을 활용하여 이미지를 크롭하는 gradio 페이지 만들기

In [3]:
FEATURES = ['read', 'smartCrops', 'tags', 'people', 'caption', 'denseCaptions', 'objects']

def request_vision(features, image_path, language, smart_crops=""):
    endpoint = f"{ENDPOINT}computervision/imageanalysis:analyze"

    query_params = {
        "api-version":"2024-02-01",
        "features":",".join(features),
        "language":language
    }
    
    if "smartCrops" in features:
        query_params.update({
            "smartcrops-aspect-ratios":smart_crops
        })

    headers = {
        "Ocp-Apim-Subscription-Key":API_KEY,
        "Content-Type":"application/octet-stream"
    }

    with open(image_path, "rb") as image_file:
        image_data = image_file.read()

    response = requests.post(endpoint, params=query_params, headers=headers, data=image_data)
    print(response.text)
    if response.status_code == 200:
        return response.json()
    else:
        return {}

def random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

def get_font():
    font_size = 50
    if platform.system() == "Darwin":
        font = ImageFont.truetype("AppleGothic.ttf", size=font_size)
    elif platform.system() == "Windows":
        font = ImageFont.truetype("malgun.ttf", size=font_size)
    else:
        font = ImageFont.load_default(size=font_size)
    
    return font

def draw_image(image_path, features, response_json):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    font = get_font()

    for feature in features:
        result_object = response_json['{}Result'.format(feature)]
        color = random_color()

        if "values" in result_object.keys():
            block_list = result_object['values']
            for block in block_list:
                bounding_box = block.get('boundingBox', None)
                confidence = block.get('confidence', 1)
                if bounding_box and confidence > 0.80:
                    x, y, w, h = bounding_box['x'], bounding_box['y'], bounding_box['w'], bounding_box['h']
                    draw.rectangle([(x, y), (x+w, y+h)], outline=color, width=5)
                    draw.text((x+10, y+10), text=feature, fill=color, font=font)

    return image

def change_image(image_path, features, language, smart_crops):
    if image_path:
        response_json = request_vision(features, image_path, language, smart_crops)
        image = draw_image(image_path, features, response_json)
        return image, response_json
    else:
        return None, None
    
def change_features(features):
    print(features)
    if "smartCrops" in features:
        return features, gr.update(visible=True)
    else:
        return features, gr.update(visible=False)

def change_language(language, features):
    print(language)
    value = list(set(FEATURES[:4]) & set(features))
    # ko를 클릭하게 되면, people까지 클릭됨. choices로 받아서 value에 저장
    if "ko" == language:
        return language, gr.update(choices=FEATURES[:4], value=value)
    else:
        return language, gr.update(choices=FEATURES, value=value)

with gr.Blocks() as demo:

    with gr.Column():
        language_radio = gr.Radio(label="언어 선택", choices=['en', 'ko'], value='en')
        features_checkbox = gr.CheckboxGroup(label="Features", choices=FEATURES)
        smart_crops_textbox = gr.Textbox(label="Smart Crops", visible=False)

    with gr.Column():
        input_image = gr.Image(label="입력 이미지", type="filepath", height=400)
        send_button = gr.Button("전송")

    with gr.Row():
        output_image = gr.Image(label="출력 이미지", type='pil', interactive=False)
        output_json = gr.JSON(label="결과 JSON")
    
    # input_image.change(change_image, inputs=[input_image, features_checkbox, language_radio, smart_crops_textbox], outputs=[output_image, output_json])
    send_button.click(change_image, inputs=[input_image, features_checkbox, language_radio, smart_crops_textbox], outputs=[output_image, output_json])
    language_radio.change(change_language, inputs=[language_radio], outputs=[language_radio, features_checkbox])
    features_checkbox.change(change_features, inputs=[features_checkbox], outputs=[features_checkbox, smart_crops_textbox])
demo.launch()

# request_vision(FEATURES, "C:/Users/jooeu/Desktop/git/ms-ai-school/250324_AIVision/test2.jpg")

c:\Users\jooeu\anaconda3\envs\ms-ai\Lib\site-packages\gradio\utils.py:1024: UserWarning: Expected 2 arguments for function <function change_language at 0x0000027E1BD4CEA0>, received 1.
  warnings.warn(
c:\Users\jooeu\anaconda3\envs\ms-ai\Lib\site-packages\gradio\utils.py:1028: UserWarning: Expected at least 2 arguments for function <function change_language at 0x0000027E1BD4CEA0>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
